# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.112314e+02     1.655468e+00
 * time: 0.09197187423706055
     1     1.034287e+01     9.727900e-01
 * time: 1.000229835510254
     2    -1.232662e+01     1.067384e+00
 * time: 1.1167609691619873
     3    -3.436391e+01     9.171943e-01
 * time: 1.282438039779663
     4    -4.794520e+01     6.917427e-01
 * time: 1.4364559650421143
     5    -5.732968e+01     2.351459e-01
 * time: 1.5852069854736328
     6    -6.006813e+01     1.696503e-01
 * time: 1.6930279731750488
     7    -6.107399e+01     6.119983e-02
 * time: 1.81392502784729
     8    -6.147364e+01     6.638063e-02
 * time: 1.9460928440093994
     9    -6.171714e+01     5.850359e-02
 * time: 2.054672956466675
    10    -6.189595e+01     2.331515e-02
 * time: 2.171311855316162
    11    -6.202054e+01     1.782125e-02
 * time: 2.288482904434204
    12    -6.207831e+01     1.659982e-02
 * time: 2.3969569206237793
    13    -6.212639e+01     1.344925e-02
 * time: 2.5144870281219482

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671077
    AtomicLocal         -18.8557681
    AtomicNonlocal      14.8522635
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485380 
    Xc                  -19.3336824

    total               -62.261666460368
